In [100]:
import requests
import pandas as pd
import pprint
import os
from dotenv import load_dotenv

In [102]:
#Lectura e importacion de datos
id_ind = {'3108001001' : 'Porcentaje analfabetismo',
          '1005000038' : 'Promedio escolaridad',
          '6200205239' : 'Poblacion con educacion basica',
          '6200205241' : 'Poblacion con bachillerato',
          '6200205242' : 'Poblacion con estudios superiores',
          '6207019020' : 'Porcentaje sin escolaridad',
          '1004000001' : 'Derechohabientes'
          }

id_estado = {
    "Aguascalientes": "01",
    "Baja California": "02",
    "Baja California Sur": "03",
    "Campeche": "04",
    "Coahuila de Zaragoza": "05",
    "Colima": "06",
    "Chiapas": "07",
    "Chihuahua": "08",
    "Ciudad de México": "09",
    "Durango": "10",
    "Guanajuato": "11",
    "Guerrero": "12",
    "Hidalgo": "13",
    "Jalisco": "14",
    "México": "15",
    "Michoacán de Ocampo": "16",
    "Morelos": "17",
    "Nayarit": "18",
    "Nuevo León": "19",
    "Oaxaca": "20",
    "Puebla": "21",
    "Querétaro": "22",
    "Quintana Roo": "23",
    "San Luis Potosí": "24",
    "Sinaloa": "25",
    "Sonora": "26",
    "Tabasco": "27",
    "Tamaulipas": "28",
    "Tlaxcala": "29",
    "Veracruz": "30",
    "Yucatán": "31",
    "Zacatecas": "32"
}

load_dotenv()
token_inegi = os.getenv("token_inegi")

indicadores = (",".join(list(id_ind.keys()))).strip()
valores = []

for estado in id_estado:
    inegi_url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicadores}/es/070000{id_estado[estado]}/false/BISE/2.0/{token_inegi}?type=json"
    response = requests.get(inegi_url)
    data = response.json()
    for indicador in data['Series']: 
        for observacion in indicador['OBSERVATIONS']:
            indicador_actual = indicador['INDICADOR']
            fecha = observacion['TIME_PERIOD']
            valor = observacion['OBS_VALUE']
            valores.append({"estado": estado, "indicador": indicador_actual, "indicador_nombre": id_ind[indicador_actual], "año" : fecha, "valor" : valor})

#print(pprint.pp(valores))
        

In [104]:
#Creacion del dataframe
df_edu = pd.DataFrame(valores)
df_edu

,estado,indicador,indicador_nombre,año,valor
0,Aguascalientes,1004000001,Derechohabientes,2000,523201.00000000000000000000
1,Aguascalientes,1004000001,Derechohabientes,2005,758160.00000000000000000000
2,Aguascalientes,1004000001,Derechohabientes,2010,930149.00000000000000000000
3,Aguascalientes,1004000001,Derechohabientes,2020,1161139.00000000000000000000
4,Aguascalientes,1005000038,Promedio escolaridad,1995,None
...,...,...,...,...,...
1083,Zacatecas,6200205242,Poblacion con estudios superiores,1990,24384.00000000000000000000
1084,Zacatecas,6200205242,Poblacion con estudios superiores,2000,49138.00000000000000000000
1085,Zacatecas,6200205242,Poblacion con estudios superiores,2010,98050.00000000000000000000
1086,Zacatecas,6207019020,Porcentaje sin escolaridad,2015,4.92437210000000000000


In [106]:
#Configuramos multi-indice por estado e indicador
df_edu.set_index(['estado','indicador'], inplace = True)
df_edu

indicador_nombre   año  \
estado         indicador                                             
Aguascalientes 1004000001                   Derechohabientes  2000   
               1004000001                   Derechohabientes  2005   
               1004000001                   Derechohabientes  2010   
               1004000001                   Derechohabientes  2020   
               1005000038               Promedio escolaridad  1995   
...                                                      ...   ...   
Zacatecas      6200205242  Poblacion con estudios superiores  1990   
               6200205242  Poblacion con estudios superiores  2000   
               6200205242  Poblacion con estudios superiores  2010   
               6207019020         Porcentaje sin escolaridad  2015   
               6207019020         Porcentaje sin escolaridad  2020   

                                                  valor  
estado         indicador                                 
Aguascalientes 1004000001   523201.00000000000000000000  
               1004000001   758160.00000000000000000000  
               1004000001   930149.00000000000000000000  
               1004000001  1161139.00000000000000000000  
               1005000038                          None  
...                                                 ...  
Zacatecas      6200205242    24384.00000000000000000000  
               6200205242    49138.00000000000000000000  
               6200205242    98050.00000000000000000000  
               6207019020        4.92437210000000000000  
               6207019020        3.93774817559726000000  

[1088 rows x 3 columns]

In [108]:
df_edu.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1088 entries, ('Aguascalientes', '1004000001') to ('Zacatecas', '6207019020')
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   indicador_nombre  1088 non-null   object
 1   año               1088 non-null   object
 2   valor             1055 non-null   object
dtypes: object(3)
memory usage: 29.4+ KB


In [110]:
#Revisamos si hay valores nulos
df_edu.isna().sum()

indicador_nombre     0
año                  0
valor               33
dtype: int64

In [112]:
#Limpieza de valores nulos
df_edu_drop = df_edu.dropna(subset = ['valor'])
df_edu_drop.isna().sum()

indicador_nombre    0
año                 0
valor               0
dtype: int64